In [9]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

In [4]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

In [6]:
import json 
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Olivia Nguyen',
  'subject': 'Introduction to Computer Science',
  'stars': 4,
  'review': 'Professor Nguyen is an excellent lecturer who makes complex topics accessible. Her assignments are challenging but fair, and she is always available to provide extra help during office hours.'},
 {'professor': 'Professor Michael Goldstein',
  'subject': 'Organic Chemistry',
  'stars': 3,
  'review': 'While Professor Goldstein is knowledgeable about the subject matter, his lectures can be dry and he does not provide much guidance on how to approach the challenging problem sets. The exams are also very difficult.'},
 {'professor': 'Dr. Sarah Patel',
  'subject': 'Art History',
  'stars': 5,
  'review': 'Dr. Patel is passionate about art and her enthusiasm is infectious. Her lectures are engaging and she encourages students to think critically about the works we study. The reading load is manageable, and the assignments are interesting and thought-provoking.'},
 {'professor': 'P

In [11]:
processed_data = []
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input=review['review'],
        model='text-embedding-3-small',
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"],
        }
    })


In [12]:
processed_data[0]

{'values': [-0.041882344,
  0.0004304235,
  -0.011563434,
  0.00697225,
  0.036411997,
  -0.02194244,
  0.0065876166,
  0.035923574,
  0.01619125,
  -0.01372471,
  -0.0025382775,
  0.040099595,
  -0.038390115,
  -0.033945456,
  -0.025251511,
  0.04857375,
  -0.04879354,
  -0.021026647,
  0.026912153,
  0.0052902405,
  0.00871226,
  0.0005269635,
  0.04000191,
  0.008999209,
  -0.033750087,
  -0.056852534,
  0.03316398,
  0.0042279186,
  0.0009073999,
  0.0029045956,
  0.03216271,
  -0.0042065503,
  -0.0023871714,
  0.014408503,
  -0.01228386,
  0.07345895,
  0.021563912,
  0.003495283,
  -0.007888045,
  -0.00840089,
  0.008706155,
  0.010379006,
  -0.030184593,
  0.019939903,
  -0.0067524593,
  0.031088177,
  -0.008455837,
  0.00666088,
  -0.017790837,
  0.057927065,
  -0.040392652,
  0.028597215,
  0.06618143,
  -0.008193309,
  -0.045154784,
  0.018645579,
  0.019182846,
  0.03775516,
  -0.0027840158,
  -0.032870922,
  0.015959248,
  -0.021160962,
  0.005564979,
  0.04061244,
  -0.056

In [14]:
index = pc.Index("rag")
index.upsert(
    vectors=processed_data,
    namespace="ns1",
)

{'upserted_count': 21}

In [15]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 21}},
 'total_vector_count': 21}